In [ ]:
from tensorflow import keras
from PIL import Image
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

RESOLUTION_WIDTH = 224
RESOLUTION_HEIGHT = 224

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

Funciones para la imagenes Cargar y Mezclar

In [ ]:
def cargar_imagenes(ruta, target_size=(RESOLUTION_WIDTH, RESOLUTION_HEIGHT)):
    X = []
    y = []
    for folder in os.listdir(ruta):
        for file in os.listdir(os.path.join(ruta, folder)):
                img = load_img(os.path.join(ruta, folder, file), target_size=target_size, color_mode='grayscale')
                img = img_to_array(img)
                X.append(img)
                y.append(0 if folder == 'no' else 1)
      
    return np.array(X), np.array(y)

def mezclar_dataset(X, y, semilla=0):
    indices = np.arange(X.shape[0])
    np.random.seed(semilla)
    np.random.shuffle(indices)
    return X[indices], y[indices]

In [ ]:
X_train, y_train = cargar_imagenes('data_2/dataset/train')
X_val, y_val = cargar_imagenes('data_2/dataset/valid')
X_test, y_test = cargar_imagenes('data_2/dataset/test')

In [ ]:
print(X_train.shape, X_val.shape, X_test.shape)
print(y_train.shape, y_val.shape, y_test.shape)

In [ ]:
X_train, y_train = mezclar_dataset(X_train, y_train)
X_val, y_val = mezclar_dataset(X_val, y_val)
X_test, y_test = mezclar_dataset(X_test, y_test)

In [ ]:
def mostrar_imagenes(X, y, n=5):
    plt.figure(figsize=(6, 6))
    for i in range(n**2):
        plt.subplot(n, n, i+1)
        plt.imshow(X[i]/255)
        if y[i] == 0:
            plt.title('No TUMOR', color='green', fontsize=5)
        else:
            plt.title('TUMOR', color='red', fontsize=5)
        plt.axis('off')
    plt.show()

mostrar_imagenes(X_train, y_train)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rescale=1./255,             # Reescalar los valores de los pixeles
    rotation_range=20,          # Rango de grados para rotar la imagen
    width_shift_range=0.2,      # Rango de fracción para trasladar la imagen en horizontal
    height_shift_range=0.2,     # Rango de fracción para trasladar la imagen en vertical
    shear_range=0.2,            # Rango de ángulo para aplicar cizalladura
    zoom_range=0.2,             # Rango de zoom
    horizontal_flip=True,       # Voltea horizontalmente las imágenes
    fill_mode='nearest'         # Estrategia de relleno para los nuevos pixeles
)


datagen_test_val = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow(X_train, y_train, batch_size=32)

val_generator = datagen_test_val.flow(X_val, y_val, batch_size=32)
test_generator = datagen_test_val.flow(X_test, y_test, batch_size=32)

CNN

In [ ]:
def crea_modelo_cnn(input_shape=(RESOLUTION_HEIGHT, RESOLUTION_WIDTH, 1)):

    model = keras.Sequential([
        keras.layers.Conv2D(64, (3, 3), activation='relu', padding = 'same', input_shape=input_shape),
        keras.layers.BatchNormalization(),
        # keras.layers.Conv2D(64, (3, 3), padding = 'same', activation='relu'),
        # keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D((2, 2)),
        
        
        keras.layers.Conv2D(128, (3, 3), padding = 'same', activation='relu'),
        keras.layers.BatchNormalization(),
        # keras.layers.Conv2D(128, (3, 3), padding = 'same', activation='relu'),
        # keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D((2, 2)),

        
        keras.layers.Conv2D(128, (3, 3), padding = 'same', activation='relu'),
        keras.layers.BatchNormalization(),
        # keras.layers.Conv2D(128, (3, 3), padding = 'same', activation='relu'),
        # keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D((3, 3), strides=2),
        
        keras.layers.Conv2D(256, (3, 3), padding = 'same', activation='relu'),
        keras.layers.BatchNormalization(),
        # keras.layers.Conv2D(256, (3, 3), padding = 'same', activation='relu'),
        # keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D((2, 2), strides=2),
        
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dropout(0.3),
        
        keras.layers.Dense(1, activation='sigmoid')   
    ])
    
    return model

model = crea_modelo_cnn()

optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
reduce_lr = keras.callbacks.ReduceLROnPlateau(patience=7, factor=0.2, min_lr=1e-6)

model.summary()

In [ ]:
history = model.fit(train_generator, 
                    epochs=200,
                    validation_data=val_generator,
                    #batch_size = 75,
                    callbacks=[early_stopping, reduce_lr])

#evaluar el modelo
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Preción con test: {round(test_accuracy*100,2)} %')

In [ ]:
model.save('modelo_cnn.keras')

In [ ]:
#evaluar el modelo
test_loss, test_acc = model.evaluate(test_generator)
print(f'Precisión con test: {round(test_acc*100,2)} %')